# Mount Drive

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


# Install CLIP 

In [2]:
# ! pip install torch==1.7.1{torch_version_suffix} torchvision==0.8.2{torch_version_suffix} -f https://download.pytorch.org/whl/torch_stable.html ftfy regex

# # # The following command installs the `clip` module from its source:
! pip install git+https://github.com/openai/CLIP.git

  Cloning https://github.com/openai/CLIP.git to /tmp/pip-req-build-lsgacphj
  Running command git clone -q https://github.com/openai/CLIP.git /tmp/pip-req-build-lsgacphj
     |████████████████████████████████| 71kB 7.2MB/s 
     |████████████████████████████████| 776.8MB 23kB/s 
     |████████████████████████████████| 12.8MB 20.3MB/s 
  Created wheel for clip: filename=clip-1.0-cp37-none-any.whl size=1368708 sha256=30c59a682beeebfabab3e04c0e85f2c2a4d1b9f43dc7880f33ac0cae632f490e
  Stored in directory: /tmp/pip-ephem-wheel-cache-81n12o2s/wheels/79/51/d7/69f91d37121befe21d9c52332e04f592e17d1cabc7319b3e09
  Created wheel for ftfy: filename=ftfy-6.0.1-cp37-none-any.whl size=41573 sha256=021f6f34f476dd35ace70c4b09ae38bbe33ad9de6d840e7693357ef15c154279
  Stored in directory: /root/.cache/pip/wheels/ae/73/c7/9056e14b04919e5c262fe80b54133b1a88d73683d05d7ac65c
Successfully built clip ftfy
ERROR: torchtext 0.9.1 has requirement torch==1.8.1, but you'll have torch 1.7.1 which is incompatible.
  F

# Import Necessary Packages

In [1]:
from torchvision import transforms
import torch
import matplotlib.pyplot as plt
from mpl_toolkits.axes_grid1 import ImageGrid
import os
import numpy as np
import torch
import clip
from tqdm.notebook import tqdm
from torchvision import transforms
import matplotlib.pyplot as plt
import cv2
import random
import os
import numpy as np
from PIL import Image
import torchvision   
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader
import sklearn
from torch import nn
from torch.nn.utils.rnn import *

cuda = torch.cuda.is_available()
print("cuda", cuda)
num_workers = 8 if cuda else 0
print(num_workers)
print("Torch version:", torch.__version__)

cuda True
8
Torch version: 1.7.1


# Load CLIP Model

In [2]:
print("Avaliable Models: ", clip.available_models())
model, preprocess = clip.load("RN50") # clip.load("ViT-B/32") #

input_resolution = model.input_resolution #.item()
context_length = model.context_length #.item()
vocab_size = model.vocab_size #.item()

print("Model parameters:", f"{np.sum([int(np.prod(p.shape)) for p in model.parameters()]):,}")
print("Input resolution:", input_resolution)
print("Context length:", context_length)
print("Vocab size:", vocab_size)

Avaliable Models:  ['RN50', 'RN101', 'RN50x4', 'ViT-B/32']
Model parameters: 102,007,137
Input resolution: tensor(224, device='cuda:0')
Context length: tensor(77, device='cuda:0')
Vocab size: tensor(49408, device='cuda:0')


# Selected classes and mapping for kinetics dataset

In [3]:
labels = ['making tea',
 'shaking head',
 'skiing slalom',
 'bobsledding',
 'high kick',
 'scrambling eggs',
 'bee keeping',
 'swinging on something',
 'washing hands',
 'laying bricks',
 'push up',
 'doing nails',
 'massaging legs',
 'using computer',
 'clapping',
 'drinking beer',
 'eating chips',
 'riding mule',
 'petting animal (not cat)',
 'frying vegetables',
 'skiing (not slalom or crosscountry)',
 'snowkiting',
 'massaging person’s head',
 'cutting nails',
 'picking fruit']

map_id = {}
i=0
for label in labels:
  map_id[label]=i
  i+=1

map_id

{'bee keeping': 6,
 'bobsledding': 3,
 'clapping': 14,
 'cutting nails': 23,
 'doing nails': 11,
 'drinking beer': 15,
 'eating chips': 16,
 'frying vegetables': 19,
 'high kick': 4,
 'laying bricks': 9,
 'making tea': 0,
 'massaging legs': 12,
 'massaging person’s head': 22,
 'petting animal (not cat)': 18,
 'picking fruit': 24,
 'push up': 10,
 'riding mule': 17,
 'scrambling eggs': 5,
 'shaking head': 1,
 'skiing (not slalom or crosscountry)': 20,
 'skiing slalom': 2,
 'snowkiting': 21,
 'swinging on something': 7,
 'using computer': 13,
 'washing hands': 8}

In [5]:
cd /content/drive/MyDrive/kinetics-downloader/TEST

/content/drive/.shortcut-targets-by-id/1hoNs0LbTnHoZWuEEYEXKCJH83Xbu67ZD/kinetics-downloader/TEST


In [6]:
import os
import pdb
ROOT = "/content/drive/MyDrive/kinetics-downloader/NEW_PROCESSED"
DEST = "/content/drive/MyDrive/kinetics-downloader/TEST"
cnt = 1
for filename in os.listdir(ROOT):
  if filename in labels:
    class_file = os.path.join(ROOT,filename)
    for random_tag in os.listdir(class_file):
      print("what is random",random_tag)
      video_name = os.path.join(class_file,random_tag)
      filename = filename.replace("_"," ")
      count = 0
      N = len(os.listdir(video_name))
      n = N//100
      selected_frames = np.arange(0,N,n).tolist()[0:100]
      for frame in os.listdir(video_name):
        if count in selected_frames:
          tmp = str(count)
          image = Image.open(os.path.join(video_name,frame))
          image = preprocess(image)
          image = torch.unsqueeze(image, 0)
          image = image.cuda()
          image_features = model.encode_image(image)
          image_features /= image_features.norm(dim=-1, keepdim=True)
          image_features = image_features.detach().cpu().numpy()
          if count==0:
            a = image_features
          else:
            a = np.vstack((a,image_features))
        count += 1
      cnt+=1    

      label = map_id[filename]*np.ones(a.shape[0])
      file_save = os.path.join(DEST,str(cnt))
      np.savez(file_save, data=a,label=label)
    cnt+=1    

what is random 1
what is random 2
what is random 3
what is random 4
what is random 5
what is random 6
what is random 7
what is random 8
what is random 9
what is random 1


ZeroDivisionError: ignored

# DEVELOP LSTM MODEL

In [ ]:
# LSTM Model
NUM_CLASSES = 18
class Model(nn.Module):
    def __init__(self, input_feature_size, embed_size, out_phoeneme, hidden_size):
        super(Model, self).__init__()
        self.layer1 = nn.Sequential(
            nn.Conv1d(input_feature_size, embed_size , kernel_size=3,padding=2),
            nn.ReLU(),
        )
        self.lstm = nn.LSTM(1024, hidden_size, num_layers=5,bidirectional=True)
        self.output1 = nn.Linear(hidden_size * 2, 256)
        self.output3 = nn.Linear(256,NUM_CLASSES)
    
    def forward(self, X, lengths):
        X_ = torch.transpose(X,0,1)
        X_ = torch.transpose(X_,1,2)
        X = self.layer1(X_)
        X = torch.transpose(X,0,2)
        X = torch.transpose(X,1,2)
        # Store the length 
        packed_X = pack_padded_sequence(X, lengths.cpu(), enforce_sorted=False)
        packed_out = self.lstm(packed_X)[0]
        out, out_lens = pad_packed_sequence(packed_out)
        out = self.output1(out)
        out = self.output3(out).log_softmax(2) 
        return out, out_lens
    
def init_weights(m):
  if type(m) == nn.Conv1d or type(m) == nn.Linear:
    torch.nn.init.xavier_normal_(m.weight.data)    